In [1]:
# Pitch Recognition Forked From https://github.com/justinsalamon/audio_to_midi_melodia
import os, sys
import math
import numpy as np
import pretty_midi as midi
from pylab import *
import librosa             # The librosa library
import librosa.display     # librosa's display module (for plotting features)
import IPython.display     # IPython's display module (for in-line audio)
import matplotlib.pyplot as plt # matplotlib plotting functions
import matplotlib.style as ms   # plotting style
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, fixed
import pitch_rec
from recorder import *

In [2]:
infile = "sing.wav"
outfile = "sing.mid"
def record():
    recorder = Recorder(channels=2)
    print("start recording")
    with recorder.open(infile, "wb") as audio_file:
        audio_file.record(duration=15)
    print("finish recording")
interact_manual(record)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

<function __main__.record()>

In [ ]:
y, sr = librosa.load(infile)
sr = 44100
bpm, beats = librosa.beat.beat_track(y=y, sr=sr)
high_mapper = np.array([67, 69, 71, 72, 74, 76])
male_mapper = np.array([60, 68, 67, 64, 62, 60])
female_mapper = np.array([69, 62, 63, 60, 67, 69])

In [ ]:
pitch_rec.audio_to_midi_melodia(infile, outfile, bpm, hop=128, smooth=0.1)

In [ ]:
data = midi.PrettyMIDI(outfile)
pitches = list(map(lambda x: x.pitch, data.instruments[0].notes))
high_pitch = high_mapper[high_mapper <= max(pitches)]
high_pitch = high_mapper[0] if len(high_pitch) == 0 else high_pitch[-1]
idx = np.where(high_mapper == high_pitch)[0][0]
tone = male_mapper[idx]
def generate_midi(arr, inst_name):
    arr = arr[np.argsort(arr[:,0]),:].tolist()
    melody = midi.PrettyMIDI()
    instrument = midi.Instrument(program=midi.instrument_name_to_program(inst_name))
    instrument.notes = list(map(lambda x: midi.Note(start=x[0], end=x[1], pitch=int(x[2]), velocity=int(x[3])), arr))
    melody.instruments.append(instrument)
    return melody

In [ ]:
def generate_first(pitch, sr=44100, am=-2.5, length=2.):
    base = 440 * 2 **((pitch-69)/12)
    F1 = base * 2**(-1/12)
    F2 = F1 * 2**(-3/12)
    F3 = F2 * 2**(7/12)
    F4 = F3 * 2**(-5/12)
    c1 = [sin(2 * pi * F1 * n) * exp(am * n) for n in np.arange(0., length, 1. / sr)] 
    c2 = [sin(2 * pi * F2 * n) * exp(am * n) for n in np.arange(0., length, 1. / sr)]
    c3 = [sin(2 * pi * F3 * n) * exp(am * n) for n in np.arange(0., length, 1. / sr)]
    c4 = [sin(2 * pi * F4 * n) * exp(am * n) for n in np.arange(0., length, 1. / sr)]
    return c1+c2+c3+c4+c1+c2+c3+c4
prog = np.array(generate_first(tone, sr))
IPython.display.Audio(data=prog, rate=sr) # press the "play" button to hear audio
librosa.output.write_wav('test.wav', prog, sr)

In [ ]:
#Am F C G
def generate_third(tone, t):
    chord1 = [tone + 9, tone + 4, tone, tone - 3, tone - 15]
    chord2 = [tone + 9, tone + 5, tone, tone - 7, tone - 19]
    chord3 = [tone + 7, tone + 4, tone, tone - 5, tone - 12]
    chord4 = [tone + 7, tone + 2, tone - 1, tone - 5, tone - 17]
    chord = [chord1, chord1, chord2, chord2, chord3, chord3, chord4, chord4] * 2
    chord = np.array(chord, dtype=np.int).flatten()
    length = chord.shape[0]
    notes = np.zeros((length, 4))
    notes[:, 2] = chord
    notes[:, 3] = 100
    notes[:, 0] = np.arange(0, length * t, t)
    notes[:, 1] = np.arange(t, (length + 1) * t, t)
    return notes
notes = generate_third(tone, 1)
melody = generate_midi(notes, "Lead 6 (voice)")
melody.write("third.mid")
IPython.display.Audio(data=melody.instruments[0].synthesize(fs=sr), rate=sr)

In [ ]:
def generate_fourth(tone, t):
    chord1 = [tone-15, tone+7, tone+2, tone, tone+7, tone+2, tone,tone+2, tone, tone-8, tone+7, tone+2, tone]
    chord2 = [tone-19, tone+7, tone+2, tone, tone+7, tone+2, tone,tone+2, tone, tone-12, tone+7, tone+2, tone]
    chord3 = [tone-12, tone+7, tone+2, tone, tone+7, tone+2, tone,tone+2, tone, tone-5, tone+7, tone+2, tone]
    chord4 = [tone-17, tone+7, tone+2, tone-1, tone+7, tone+2, tone-1,tone+2, tone-1, tone-5,tone+7, tone+2, tone-1]
    chord = [chord1, chord2, chord3, chord4]
    col_len = len(chord)
    row_len = len(chord[0])
    chord = np.array(chord, dtype=np.int).flatten()
    length = chord.shape[0]
    notes = np.zeros((length, 4))
    notes[:, 2] = chord
    notes[:, 3] = 100
    timeline = {
        0: [0, 2],
        1: [1, 1 + 0.75],
        2: [1, 1 + 0.75],
        3: [1, 1 + 0.75],
        4: [1 + 0.75, 2.25],
        5: [1 + 0.75, 2.25],
        6: [1 + 0.75, 2.25],
        7: [2 + 0.25, 3],
        8: [2 + 0.25, 3],
        9: [2.5, 4.25],
        10: [3, 4],
        11: [3, 4],
        12: [3, 4]
    }
    line = np.arange(length).tolist()
    line = list(map(lambda x: [4 * (x // row_len) + i for i in timeline[x % row_len]], line))
    line = np.array(line) * 0.75
    notes[:, [0, 1]] = line
    return notes
notes = generate_fourth(tone, 1)
melody = generate_midi(notes, "Acoustic Guitar (Nylon)")
melody.write("fourth.mid")
IPython.display.Audio(data=melody.instruments[0].synthesize(fs=sr), rate=sr)